In [4]:
import os

In [5]:
%pwd

'c:\\Users\\asus\\Github Projects\\Chicken_Disease_Detection\\research'

In [6]:
os.chdir("../")

In [7]:
%pwd

'c:\\Users\\asus\\Github Projects\\Chicken_Disease_Detection'

In [22]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_dir: Path
    unzip_dir : Path

In [23]:
from Chicken_Disease_Classification.constants import *
from Chicken_Disease_Classification.utils.common import read_yaml, create_directories

In [42]:
#Making the Configuration manager to create a path by calling fn from common.py
class ConfigurationManager:
    """This class is designed to read configuration and parameter files from YAML files, 
    create necessary directories, and provide access to 
    specific configuration values."""
    def __init__(self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_roots])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        
        #This is to define the config variable with specific dtype
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir  = config.unzip_dir
        )

        return data_ingestion_config


In [26]:
import os
import urllib.request as request
import zipfile
from Chicken_Disease_Classification import logger
from Chicken_Disease_Classification.utils.common import get_size


In [39]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with the following info:\n{header}")
        else:
            logger.info(f"File Already Exists of size {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_file:
            zip_file.extractall(unzip_path)


In [43]:
try:
    config_manager = ConfigurationManager()
    data_ingestion_config = config_manager.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2023-11-02 09:55:36,278: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-02 09:55:36,287: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-02 09:55:36,290: INFO: common: created directory at: artifacts]
[2023-11-02 09:55:36,294: INFO: common: created directory at: artifacts/data_ingestion]


TypeError: __init__() got an unexpected keyword argument 'source_URL'